# 0. Khai báo thư viện

In [1]:
import pandas as pd
from scipy import stats

# 1. Đọc dữ liệu từ file CSV 

In [2]:
df = pd.read_csv("https://s3.ap-southeast-1.amazonaws.com/learn.codegym.vn/media/public/2021/12/31020712/subset-covid-data.csv", encoding = "ISO-8859-1")
df.head()

,country,continent,date,day,month,year,cases,deaths,country_code,population
0,Afghanistan,Asia,2020-04-12,12,4,2020,34,3,AFG,37172386.0
1,Albania,Europe,2020-04-12,12,4,2020,17,0,ALB,2866376.0
2,Algeria,Africa,2020-04-12,12,4,2020,64,19,DZA,42228429.0
3,Andorra,Europe,2020-04-12,12,4,2020,21,2,AND,77006.0
4,Angola,Africa,2020-04-12,12,4,2020,0,0,AGO,30809762.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206 entries, 0 to 205
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   country       206 non-null    object 
 1   continent     206 non-null    object 
 2   date          206 non-null    object 
 3   day           206 non-null    int64  
 4   month         206 non-null    int64  
 5   year          206 non-null    int64  
 6   cases         206 non-null    int64  
 7   deaths        206 non-null    int64  
 8   country_code  202 non-null    object 
 9   population    202 non-null    float64
dtypes: float64(1), int64(5), object(4)
memory usage: 16.2+ KB


# 2. Có mối tương quan nào giữa dân số và số ca nhiễm bệnh hay không

- Giả thuyết không: Không có mối tương quan tuyến tính giữa hai biến
- Giả thuyết đối: Không có mối tương quan tuyến tính giữa hai biến

In [4]:
df1 = df.loc[:, ["population", "cases"]]
df1.dropna(inplace = True)

In [5]:
stats.pearsonr(df1.cases, df1.population)

(0.17553926594294003, 0.01246031070420893)

Nhận xét: do pvalue = 0.012 < 0.05 nên bác bỏ giải thuyết không, chấp nhận giải thuyết đối

# 3. Có mối tương quan nào giữa số ca mắc (cases) và số ca tử vong (deaths) hay không

Do số ca nhiễm (cases) và số ca tử vong (deaths) đều là thuộc tính định lượng nên sử dụng kiểm định pearson
- Giả thuyết không: Không có mối tương quan tuyến tính giữa hai biến
- Giả thuyết đối: Không có mối tương quan tuyến tính giữa hai biến

In [6]:
df2 = df.loc[:, ["cases", "deaths"]]
df2.dropna(inplace = True)

In [7]:
stats.pearsonr(df2.cases, df2.deaths)

(0.9432201374746727, 1.3451948745846463e-99)

Nhận xét: do pvalue ~ 0 < 0.05 nên bác bỏ giả thuyết không, chấp nhận giả thuyết đối

# 4. Hãy tiến hành rời rạc hóa thuộc tính population thành các nhóm theo thứ tự tăng dần đặt tên là biến population_ordinal:
- 1: nếu population < tứ phân vị thứ nhất
- 2: Population nằm trong khoảng từ tứ phân vị thứ nhất tới tứ phân vị thứ 2
- 3: nếu population nằm trong khoảng từ tứ phân vị thứ 2 tới thứ 3
- 4: nếu population >= tứ phân vị thứ 3
Hãy kiểm tra mối tương quan giữa thuộc tính population_ordinaly với số ca nhiễm bệnh

In [8]:
# Tiến hành tính các khoảng tứ phân vị của population
q1, q2, q3 = df1.population.quantile(0.25), df1.population.quantile(0.5), df1.population.quantile(0.75)

In [9]:
# Tiến hành biến đổi population
def population_order(population):
    if population < q1:
        return 1
    elif q1 <= population < q2:
        return 2
    elif q2 <= population < q3:
        return 3
    else:
        return 4

df1["population_ordinal"] = df1.population.apply(population_order)
df1.head()

,population,cases,population_ordinal
0,37172386.0,34,4
1,2866376.0,17,2
2,42228429.0,64,4
3,77006.0,21,1
4,30809762.0,0,4


Tiến hành kiểm định: Do population_ordinal là dữ liệu kiểu thứ bậc nên sử dụng kiểm định spearman để kiểm định tương quan giữa population_ordinal và cases
- Giả thuyết không: Giữa population_ordinal và case không có tương quan
- Giả thuyết đối: Giữa population_ordinal và case có tương quan

In [10]:
stats.spearmanr(df1.cases, df1.population_ordinal)

SpearmanrResult(correlation=0.49498466493711596, pvalue=6.982786622314858e-14)